In [4]:
#Load necessary libraries
import sys
import os
import ee
import numpy as np
import pandas as pd
import random
import json
import time


In [5]:

#Initialize earth engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

Define seed number to reproduce results

In [6]:
num_seed = 30
random.seed(num_seed)

Load one image to get the projection and boundaries

In [12]:
# load one of the predictor variable images
projectionImage = ee.Image("projects/wri-datalab/CarbonSequestrationAI/PredictorVariables/PredictorVariablesCookPatton2020/wc_v2_bio_30s_05")

# save projection information
projection_ee = projectionImage.projection()
projection = projection_ee.getInfo()
crs = projection.get('crs')
crsTransform = projection.get('transform')
scale = projection_ee.nominalScale().getInfo()
print('CRS and Transform: ',crs, crsTransform)

region = projectionImage.geometry().bounds()


CRS and Transform:  EPSG:4326 [0.008333333333333333, 0, -180, 0, -0.008333333333333333, 90]


Define predictor variables

In [28]:
predictorVariableCollection = ee.ImageCollection('projects/wri-datalab/CarbonSequestrationAI/PredictorVariables/PredictorVariablesCookPatton2020')
predictorVariableImage = ee.ImageCollection(predictorVariableCollection).toBands()
bands = predictorVariableImage.bandNames().getInfo()
bands = [x for x in bands if 'Biomes' not in x]

#predictorVariableImage = predictorVariableImage.select(bands)
predictorVariableImage = predictorVariableImage.unmask(-1.7e+308)
print(len(predictorVariableImage.bandNames().getInfo()))
print(predictorVariableImage.bandNames().getInfo())



59
['BDRLOG_M_1km_ll_b1', 'BDTICM_M_1km_ll_b1', 'BLDFIE_M_sl_b1', 'BiomesMask_b1', 'CECSOL_M_sl_b1', 'CLYPPT_M_sl_b1', 'CM10_1975H_Bio20_V1_2_b1', 'CM10_1975H_Bio21_V1_2_b1', 'CM10_1975H_Bio22_V1_2_b1', 'CM10_1975H_Bio23_V1_2_b1', 'CM10_1975H_Bio24_V1_2_b1', 'CM10_1975H_Bio25_V1_2_b1', 'CM10_1975H_Bio26_V1_2_b1', 'CM10_1975H_Bio27_V1_2_b1', 'CM10_1975H_Bio28_V1_2_b1', 'CM10_1975H_Bio29_V1_2_b1', 'CM10_1975H_Bio30_V1_2_b1', 'CM10_1975H_Bio31_V1_2_b1', 'CM10_1975H_Bio32_V1_2_b1', 'CM10_1975H_Bio33_V1_2_b1', 'CM10_1975H_Bio34_V1_2_b1', 'CM10_1975H_Bio35_V1_2_b1', 'CRFVOL_M_sl_b1', 'GMTEDAspect_b1', 'GMTEDElevation_b1', 'GMTEDHillShade_b1', 'GMTEDSlope_b1', 'NHx_avg_dep_1980_2009_b1', 'NOy_avg_dep_1980_2009_b1', 'OCDENS_M_sl_b1', 'OCSTHA_M_sd_b1', 'ORCDRC_M_sl_b1', 'PHIHOX_M_sl_b1', 'PHIKCL_M_sl_b1', 'SLTPPT_M_sl_b1', 'SNDPPT_M_sl_b1', 'WWP_M_sl_b1', 'ai_et0_b1', 'et0_yr_b1', 'shortwave_radiaton_1982_2015_b1', 'wc_v2_bio_30s_01_b1', 'wc_v2_bio_30s_02_b1', 'wc_v2_bio_30s_03_b1', 'wc_v2_bio_

Sample predictor variables at training and test point locations

In [26]:
#Define properties to export to an Earth Engine asset
locationDescription = '{}'
locationAssetID = 'projects/wri-datalab/CarbonSequestrationAI/TrainingPoints/{}'
pointsAssetID = 'projects/wri-datalab/CarbonSequestrationAI/TrainingPoints/{}wCovar'

# Load the point locations assets
trainingLocationsAsset = ee.FeatureCollection(locationAssetID.format('Train'))
testLocationsAsset = ee.FeatureCollection(locationAssetID.format('Test'))

# Sample predictor variable at location
trainingPoints = predictorVariableImage.sampleRegions(trainingLocationsAsset, 
                                                         projection=projection_ee, geometries=True,tileScale=16)
testPoints = predictorVariableImage.sampleRegions(testLocationsAsset, 
                                                     projection=projection_ee, geometries=True,tileScale=16)

# # Export results
export_results_task = ee.batch.Export.table.toAsset(
    collection=trainingPoints, 
    description = locationDescription.format('Train'), 
    assetId = pointsAssetID.format('Train'))
export_results_task.start()

export_results_task = ee.batch.Export.table.toAsset(
    collection=testPoints, 
    description = locationDescription.format('Test'), 
    assetId = pointsAssetID.format('Test'))
export_results_task.start()

#Wait for last export to finish
while export_results_task.active():
    print('Polling for task (id: {}).'.format(export_results_task.id))
    time.sleep(30)
print('Done with export.')

Polling for task (id: L5THS27AFMACI4ASXMMLXYB2).
Polling for task (id: L5THS27AFMACI4ASXMMLXYB2).
Polling for task (id: L5THS27AFMACI4ASXMMLXYB2).
Polling for task (id: L5THS27AFMACI4ASXMMLXYB2).
Polling for task (id: L5THS27AFMACI4ASXMMLXYB2).
Polling for task (id: L5THS27AFMACI4ASXMMLXYB2).
Polling for task (id: L5THS27AFMACI4ASXMMLXYB2).
Polling for task (id: L5THS27AFMACI4ASXMMLXYB2).
Done with export.


In [27]:
# # Export results
bucket = 'wri-carbon-sequestration'
description = '{}_gcloud_export'
fileNamePrefix = '{}wCovar'

#Export.table.toCloudStorage(collection, description, bucket, fileNamePrefix, fileFormat, selectors)

trainingPoints = ee.FeatureCollection(pointsAssetID.format('Train'))
testPoints = ee.FeatureCollection(pointsAssetID.format('Test'))

export_results_task = ee.batch.Export.table.toCloudStorage(
    collection=trainingPoints, 
    description = description.format('Train'), 
    bucket = bucket,
    fileNamePrefix = fileNamePrefix.format('Train'),
    fileFormat = 'CSV')

export_results_task.start()

export_results_task = ee.batch.Export.table.toCloudStorage(
    collection=testPoints, 
    description = description.format('Test'), 
    bucket = bucket,
    fileNamePrefix = fileNamePrefix.format('Test'),
    fileFormat = 'CSV')
export_results_task.start()


#Wait for last export to finish
while export_results_task.active():
    print('Polling for task (id: {}).'.format(export_results_task.id))
    time.sleep(30)
print('Done with export.')

Polling for task (id: KMDI62SVPOB7UU65HEEEQTJK).
Polling for task (id: KMDI62SVPOB7UU65HEEEQTJK).
Polling for task (id: KMDI62SVPOB7UU65HEEEQTJK).
Done with export.
